# **ThinCurr_RWM class Example**

### *Files Needed*
    # wall_h5='DIIID-9_18_24.h5'
    # plasma_h5='D3D_n1.h5'
    # xml_coils='oft_in_none.xml
    # floops='floops.loc'

### *To-do*
    # show floops.loc creation
    # _____
    # functionality for making sensors?
    # __init__ and compute_sM_s has too many prints. flux_sens_eig weird prints.
    # R_inv stuff (raise errors, make workflow make sense), print statements
    # new descriptions, make sure im clearing things when necessary

# CHANGE THE os.environment

In [58]:
import sys
from typing import Union
import os
import h5py
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import matplotlib as mpl
from matplotlib import ticker
import pyvista
import scipy
from scipy.sparse.linalg import eigs
from scipy.linalg import lu_factor, lu_solve
plt.rcParams['figure.figsize']=(6,6)
plt.rcParams['font.weight']='bold'
plt.rcParams['axes.labelweight']='bold'
plt.rcParams['lines.linewidth']=2
plt.rcParams['lines.markeredgewidth']=2
%matplotlib inline
%config InlineBackend.figure_format = "retina"

os.environ['OFT_ROOTPATH'] = '/home/smiller/ThinCurr_Build/install_release'

thincurr_python_path = os.getenv('OFT_ROOTPATH')
if thincurr_python_path is not None:
    sys.path.append(os.path.join(thincurr_python_path,'python'))
from OpenFUSIONToolkit.ThinCurr import ThinCurr
from OpenFUSIONToolkit.ThinCurr.sensor import Mirnov, save_sensors
from OpenFUSIONToolkit.util import build_XDMF

In [21]:
class ThinCurr_RWM:

    def __init__(self, wall_h5, xml_coils, plasma_h5, floops, nthreads=4):

        # walls
        self.tw_wall = ThinCurr(nthreads)
        self.tw_wall.setup_model(mesh_file=wall_h5, xml_filename=xml_coils)
        # plasma
        self.tw_plasma = ThinCurr(nthreads)
        self.tw_plasma.setup_model(mesh_file=plasma_h5,xml_filename='oft_in_none.xml')           #  oft_in_none _____
        self.plasma_h5=plasma_h5
        self.floops = floops

        self.rho = None
        self.R_inv = None
        self.tau = None
        self.mat_li = None
        self.sL_tot = None
        self.eig_vals = None
        self.eig_vecs = None
        self.sM_s = None

    def resist_induct_drive(self):
        # wall calcs
        self.tw_wall.compute_Mcoil()
        self.tw_wall.compute_Lmat()
        self.tw_wall.compute_Rmat(copy_out=True)
                

        # plasma calcs _____ can redo this...
        with h5py.File(self.plasma_h5, 'r+') as h5_file:
            self.mode_drive = np.asarray(h5_file['thincurr/driver'])      # specific from DCON generation and conversion? Specific for RWM. What was the DCON to this process?*****
        self.mode_drive_full = np.zeros((2,self.tw_plasma.nelems))
        
        if self.tw_plasma.n_vcoils > 0:
            self.mode_drive_full[:,:-self.tw_plasma.n_vcoils] = self.mode_drive
        else:
            print('\n0 vcoils?????\n')
            self.mode_drive_full = self.mode_drive
            
        self.tw_plasma.compute_Mcoil()
        self.tw_plasma.compute_Lmat()
        self.tw_plasma.compute_Rmat(copy_out=True)  # _____  

    
    # compute plasma reluctance matrix rho. Modifies vacuum mutual inductance matrices
    def compute_rho(self, alpha: Union[float, int], s: Union[float, int]): 

        if self.mat_li is None:
            print('L_p not calculated: use get matrices')
            return
                
        L_w, L_c, L_p, M_wp, M_pw, M_wc, M_cw, M_pc, M_cp, R = self.mat_li        
        
        P = np.zeros(L_p.shape, dtype=np.complex128)
        for j in range(L_p.shape[0]):
            if s!=0 or alpha!=0:
                P[j,j] = -1/complex(s,alpha)
            else:
                print('/0,0 :  s,alpha =', s, alpha)
                P[j,j] = 1E20
        self.rho = np.linalg.inv(L_p) @ (P-np.identity(L_p.shape[0]))    
        
        return self.rho

    # Compute script L. This is the total effective inductance matrix 
    def compute_sL(self):

        if self.rho is None:
            print('No rho value stored') # _____raise exception?
            return
        else: 
            rho=self.rho
            
        L_w, L_c, L_p, M_wp, M_pw, M_wc, M_cw, M_pc, M_cp, R = self.mat_li
    
        sL_w  = L_w  + M_wp @ (rho @ M_pw)   # parenthesis around last two terms for speed
        sL_wc = M_wc + M_wp @ (rho @ M_pc)
        sL_wd = M_wp + M_wp @ (rho @ L_p)
        sL_cw = M_cw + M_cp @ (rho @ M_pw)
        sL_c  = L_c  + M_cp @ (rho @ M_pc)
        sL_cd = M_cp + M_cp @ (rho @ L_p)
        sL_dw = M_pw + L_p  @ (rho @ M_pw)
        sL_dc = M_pc + L_p  @ (rho @ M_pc)
        rho_by_L_p  =  rho @ L_p
        sL_d  = L_p @ (np.identity(L_p.shape[0]) + rho_by_L_p)
        
        row1 = np.hstack((sL_w, sL_wc, sL_wd))
        row2 = np.hstack((sL_cw, sL_c, sL_cd))
        row3 = np.hstack((sL_dw, sL_dc, sL_d))
        self.sL_tot = np.vstack((row1, row2, row3))

        self.tau = None
        
        return self.sL_tot

    # combine the resistance matrices
    def _compute_R(self, R_w,R_c,R_p):
        R = scipy.linalg.block_diag(R_w, R_c, R_p)
        self.R_inv = None
        self.tau = None
        return R

    # voltage _____
    
    def get_matrices(self):

        tw_wall = self.tw_wall
        tw_plasma = self.tw_plasma
        mode_drive_full = self.mode_drive_full
        mode_drive = self.mode_drive   
        
        # wall matrices
        L_w = tw_wall.Lmat[:-tw_wall.n_vcoils,:-tw_wall.n_vcoils]
        R_w = tw_wall.Rmat[:-tw_wall.n_vcoils,:-tw_wall.n_vcoils]
        # Coil matrices
        L_c = tw_wall.Lmat[-tw_wall.n_vcoils:,-tw_wall.n_vcoils:]
        R_c = tw_wall.Rmat[-tw_wall.n_vcoils:,-tw_wall.n_vcoils:]
        # Wall,Coil Matrix
        M_cw = tw_wall.Lmat[-tw_wall.n_vcoils:,:-tw_wall.n_vcoils]
        #M_wc2 = tw_wall.Lmat[:-tw_wall.n_vcoils,-tw_wall.n_vcoils:]
        M_wc = M_cw.T  # check M_cw = tw_wall.Lmat[:-tw_wall.n_vcoils,-tw_wall.n_vcoils:].T

        
        # Plasma matrices
        L_p_full = tw_plasma.Lmat[:-tw_plasma.n_vcoils,:-tw_plasma.n_vcoils]
        R_p_full = tw_plasma.Rmat[:-tw_plasma.n_vcoils,:-tw_plasma.n_vcoils]
        # Plasma,Coil Matrix
        M_cp_full = tw_plasma.Lmat[-tw_plasma.n_vcoils:,:-tw_plasma.n_vcoils]
        
        print(L_p_full.shape, tw_plasma.Lmat.shape, tw_plasma.n_vcoils, mode_drive.shape)
        
        qwerty = L_p_full @ mode_drive.T
        L_p = mode_drive @  qwerty
        R_p = mode_drive @ (R_p_full @ mode_drive.T)
        M_cp = M_cp_full @ mode_drive.T
        M_pc = M_cp.T
        #M_pc_full = tw_plasma.Lmat[:-tw_plasma.n_vcoils,-tw_plasma.n_vcoils:]
        #M_pc2 = np.dot(mode_drive,M_pc_full)
        M_pw_full = tw_plasma.cross_eval(tw_wall,mode_drive_full) # M_wp = M_w_p_full * modes^T
        M_pw = M_pw_full[:,0:-tw_wall.n_vcoils]
        M_wp = M_pw.T
        
        R = self._compute_R(R_w,R_c,R_p)
        self.mat_li = [L_w, L_c, L_p, M_wp, M_pw, M_wc, M_cw, M_pc, M_cp, R]
        return self.mat_li
        

    # this compute_tau might be useless... doesnt  make sense ww/ how self.R_inv works
    def compute_tau(self):
        if self.sL_tot is None:
            print('you have to calc sL')
            return
        if self.R_inv is None:
            print('you have to calc R_inv')
            return
            
        
        self.tau = -np.dot(self.R_inv, self.sL_tot)
        return self.tau
    
    def compute_current_eigs(self, k=1, which_1='LR'):#, which_2='SR'):

        if self.R_inv is None:
            self.R_inv = np.linalg.inv(self.mat_li[-1])
        if self.tau is None:
            self.tau = -np.dot(self.R_inv, self.sL_tot)
        
        self.eig_vals, self.eig_vecs = eigs(self.tau, k=k, which=which_1 )

        return self.eig_vals, self.eig_vecs

    
    def save_curr_eigs(self, save_eigs_num): 
        if self.eig_vecs is None:
            print('no eig_vecs stored')
            return
        eig_vecs = self.eig_vecs
        if save_eigs_num > eig_vecs.shape[1]:
            print(f'only {eig_vecs.shape[1]} eigs are stored')
            return
        
        eig_vecs2 = eig_vecs.T[:,0:self.tw_wall.nelems] #np vs nelems
        r_evs = np.real(eig_vecs2)
        i_evs = np.imag(eig_vecs2)
        
        for n in range(save_eigs_num):
            self.tw_wall.save_current(r_evs[n,:], f'Jr_{n}_wall')
            self.tw_wall.save_current(i_evs[n,:], f'Ji_{n}_wall')
        build_XDMF(path='.') # _____ does this work
    

    # sensor stuff    #_____ doesnt need M_cp
    def _M_x_to_sM_s(self, M_sw, M_sp, M_sc, M_pw, M_cp, L_p, rho, M_pc):
    
        sM_sw = M_sw + M_sp @ (rho @ M_pw)
        sM_sc = M_sc + M_sp @ (rho @ M_pc)
        sM_sd = M_sp @ (np.identity(L_p.shape[0]) + rho @ L_p)
        sM = np.hstack((sM_sw, sM_sc, sM_sd))
        
        return sM

    def compute_sM_s(self):
        if self.mat_li is None:
            print('Error: Matrices not computed from get_matrices')
            return
        if self.rho is None:
            print('Error: Rho not computed from compute_rho')
            return
        
        M_sw_full, M_sc_icoil, _ = self.tw_wall.compute_Msensor(self.floops) 
        M_sp_full, _, _ = self.tw_plasma.compute_Msensor(self.floops)     # M_pc M_sc based on vcoils, icoils respectively?
        
        M_sw = M_sw_full.T[:,0:-self.tw_wall.n_vcoils]  
        M_sc = M_sp_full[-self.tw_plasma.n_vcoils:, :].T  
        M_sp = np.dot(self.mode_drive, M_sp_full[0:-self.tw_plasma.n_vcoils, :]).T

        
        L_w, L_c, L_p, M_wp, M_pw, M_wc, M_cw, M_pc, M_cp, R = self.mat_li
        rho = self.rho

        self.sM_s = self._M_x_to_sM_s(M_sw, M_sp, M_sc, M_pw, M_cp, L_p, rho, M_pc) 
        
        return self.sM_s
    
    def flux_sens_eig(self, t_vals): 
        
        if self.eig_vecs is None or self.eig_vals is None:
            print('no eig_vecs stored')
            return
            
        eig_vecsT = self.eig_vecs.T
        I_tot = np.zeros((eig_vecsT.shape[1], t_vals.shape[0]), dtype='complex128')   # total elems by time # self.tw_wall.nelems_____
                              
        for j,eig_val in enumerate(self.eig_vals):
            I_tot += eig_vecsT[j][:, np.newaxis] * np.exp(eig_val * t_vals)
            if eig_val>0: print(f'\n{j}, {eig_val}\n')
            print(f'{j+1}/{self.eig_vals.shape[0]}',  end='\r')

        self.flux_sens = self.sM_s @ I_tot
        return self.flux_sens

    def flux_sens_imp(self, t_vals, I0=None): 

        if self.tau is None or self.sM_s is None:
            print('no tau or sM_s calculated')
            return
        
        if I0 is None:
            I0 = np.zeros((self.tw_wall.nelems, 0), dtype='complex128')
    
        
        n_steps = len(t_vals)

        I_tot = np.zeros((I0.shape[0], n_steps), dtype=complex)
        I_tot[:, 0] = I0
        I_current = I0.copy()
    
        for i in range(n_steps - 1):
            dt = t_vals[i+1] - t_vals[i]
            # implicit backward Euler update:
            #   (I - dt * A) I_next = I_current
            I_current = np.linalg.solve(np.eye(self.tau.shape[0]) - dt * self.tau, I_current)
            I_tot[:, i+1] = I_current

            # print(f"Time step {i+1}/{n_steps-1}", end="\r")
        
        flux_sens = self.sM_s @ I_tot
        self.flux_sens = flux_sens
        return flux_sens

In [5]:
RWM = ThinCurr_RWM(wall_h5='DIIID-9_18_24.h5', plasma_h5='D3D_n1.h5', xml_coils='oft_in_none.xml', floops='floops.loc')

#----------------------------------------------
Open FUSION Toolkit Initialized
Development branch:   main
Revision id:          66f38de
Parallelization Info:
  Not compiled with MPI
  # of OpenMP threads =    2
Fortran input file    = oftpyin                                                                         
XML input file        = none                                                                            
Integer Precisions    =    4   8
Float Precisions      =    4   8  10
Complex Precisions    =    4   8
LA backend            = native
#----------------------------------------------


Creating thin-wall model
  Ensuring surface normal orientations
    Chunk oriented       1   28806
 Orientation depth =       16548
  Loading V(t) driver coils

 Coils set definitions
 Found       1  coil sets
   Set  :            1
     nCoils  :            1

  Loading I(t) driver coils

  # of points    =        15271
  # of edges     =        44203
  # of cells     =        28806
  # of 

In [7]:
RWM.resist_induct_drive()

 Building coil<->element inductance matrices
     Time =  0s          
 Building coil<->coil inductance matrix
  Vcoil    1: L [H] =   7.6842E-05
 Building element<->element self inductance matrix
     Time =  4m 13s      
 Building resistivity matrix
  Vcoil    1: R [Ohm] =   3.7699E-05
 Building coil<->element inductance matrices
     Time =  0s          
 Building coil<->coil inductance matrix
  Vcoil    1: L [H] =   7.6842E-05
 Building element<->element self inductance matrix
     Time = 25s          
 Building resistivity matrix
  Vcoil    1: R [Ohm] =   3.7699E-05


In [9]:
matrices = RWM.get_matrices()

(3201, 3201) (3202, 3202) 1 (2, 3201)
 Applying MF element<->element inductance matrix


[    0] WARNING: V-coil contributions were not computed.
[    0] WARNING: V-coil contributions were not computed.


     Time =  4m 47s      


In [11]:
alpha,s = 1,1
rho = RWM.compute_rho(alpha, s)

In [13]:
sL = RWM.compute_sL()

In [27]:
tau = RWM.compute_current_eigs()

In [29]:
RWM.save_curr_eigs(1)

Saving vector plot field: Jr_0_wall
Saving vector plot field: Jr_0_wall_v
Saving scalar plot field: Jr_0_wall_p
Saving vector plot field: Ji_0_wall
Saving vector plot field: Ji_0_wall_v
Saving scalar plot field: Ji_0_wall_p
Removing old Xdmf files
Creating output files


In [42]:
sM_s = RWM.compute_sM_s()


 Loading floop information:
   # of floops =         120
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =           5
     # of floops pts =          

In [44]:
t_vals = np.linspace(0, 1e4, int(3000*1e4/1e4))
flux_sens_eig = RWM.flux_sens_eig(t_vals)
print(flux_sens_eig)


0, (0.0061612195450094-0.006762911032365764j)

1/3
1, (0.005823285605504777-0.006234651747738089j)



In [50]:
flux_sens_imp = RWM.flux_sens_imp(t_vals, I0 = np.zeros(13413))
print(flux_sens_eig, '\n', flux_sens_imp)

KeyboardInterrupt: 

#
#
#
#
## **Other code that was not added to the class**
## Mostly sensor position code
#
#

In [7]:
# find the positions in RZ coordinates of where the sensorscould go 
# returns r and z of points for 2D cross section
# Computer sensor mutual inductance matrices

def rz_sensor_positions(mesh, scaling_factor = 0.98, nodeset_num = 1, num_sens_step = 1):
    # 120 total pointsin 2D cross section
    # num_sens_step is the fraction 1/x of these points we want to use (poloidal direction)***** change
    
    # extract 2D cross  section
    with h5py.File(mesh, 'r') as hdf:
        R = np.array(hdf['mesh/R'])
        NODESET = np.array(hdf[f'mesh/NODESET000{nodeset_num}'])
    coords = R[NODESET-1]
    coords[:,1]= 0

    # make a smaller version
    centroid = np.mean(coords, axis=0)
    translated_points = coords - centroid
    scaled_points = translated_points * scaling_factor
    scaled_shape = scaled_points + centroid

    r = scaled_shape[:, 0][::num_sens_step]
    z = scaled_shape[:, 2][::num_sens_step]

    return r,z,centroid
    
# changes the rz coordinates to xyz coordinates
def rz_to_xyz(r,z_vals,centroid,num_theta, cn=True):
    # num_theta is the number of sensors we want in the theta direction (toroidally)
    # cn is centroid normal, cn=False means it has a pn (poloidal normal)
    
    theta = np.linspace(0, 2 * np.pi, num_theta, endpoint=False) # evenly spaced theta
    print(theta.shape)
    xyz = np.zeros((r.shape[0],num_theta,6))
    print(xyz.shape)

    for i in range(len(r)):
        x = r[i] * np.cos(theta)
        y = r[i] * np.sin(theta)
        z = np.full_like(theta, z_vals[i])  # z remains constant for the current r, z pair

        cent_r = (centroid[0]**2+centroid[1]**2)**0.5
        cent_z = centroid[2]

        nx = (r[i]-cent_r) * np.cos(theta)
        nx = nx / np.linalg.norm(nx)
        ny = (r[i]-cent_r) * np.sin(theta)
        nz = np.full_like(theta, z_vals[i])-centroid[2]  # z remains constant for the current r, z pair
        
        xyz[i] = np.column_stack((x, y, z, nx, ny, nz))           
    
        '''x2 = (r[i]+c_len) * np.cos(theta)
        y2 = (r[i]+c_len) * np.sin(theta)
        z_val2 = np.full_like(theta, z[i])

        x3 = (r[i]+c_len) * np.cos(theta)
        y3 = (r[i]+c_len) * np.sin(theta)
        z_val3 = np.full_like(theta, z[i]+c_len)

        x4 = r[i] * np.cos(theta)
        y4 = r[i] * np.sin(theta)
        z_val4 = np.full_like(theta, z[i]+c_len)'''

    print(xyz.shape)
    if cn == False:
        xyz2 = xyz.copy()
        for i,rz in enumerate(xyz2):
            for j,pt in enumerate(rz):
                pos = pt[0:3]
                cross1 = pt[3:6]
                #print(pos.shape,cross1.shape,pt.shape)
                cross2 = np.cross(pos, cross1)
                normal = np.cross(cross2, cross1)
                normal = normal / np.linalg.norm(normal)
                xyz[i][j][3:6] = normal
    else:
        xyz2 = xyz.copy()
        for i,rz in enumerate(xyz2):
            for j,pt in enumerate(rz):
                pos = pt[0:3]
                normal = pt[3:6]
                #print(pos.shape,cross1.shape,pt.shape)
                #cross2 = np.cross(pos, cross1)
                #normal = np.cross(cross2, cross1)
                normal = normal / np.linalg.norm(normal)
                xyz[i][j][3:6] = normal
            
    print(xyz.shape)
    return xyz   # z is a 3d array. First dim is poloidal? direction of points, second dim is toroidal? direction ofpoints, third dim is the 4 x,y,z var of the coil


# example sensor uses

r,z,centroid = rz_sensor_positions('DIIID-9_18_24.h5',num_sens_step=120)
xyz = rz_to_xyz(r,z,centroid,120,cn=True)
print(xyz.shape, '\n')

np.savez('xyz_1by120_cn_n.npz', xyz=xyz)
xyz = rz_to_xyz(r,z,centroid,120,cn=False)
np.savez('xyz_1by120_pn_n.npz', xyz=xyz)

# save the sensors as mirnov coils 
xyz = np.load('xyz_1by120_pn_n.npz')['xyz']
sensors = []

for rz_dim,line in enumerate(xyz):
    for theta_dim,pt in enumerate(line):
        
        p1 = pt[0:3]  
        p2 = pt[3:6] 
        '''p3 = pt[6:9] 
    
        v1 = p2 - p1
        v2 = p3 - p1 
        
        normal = np.cross(v1, v2)'''

        sensors.append(Mirnov(p1, p2, f'{rz_dim},{theta_dim}'))
save_sensors(sensors)



M_sw_full, M_sc_icoil, _ = tw_wall.compute_Msensor('floops.loc') 
M_sp_full, _, _ = tw_plasma.compute_Msensor('floops.loc')     # M_pc M_sc based on vcoils, icoils respectively?

M_sw = M_sw_full.T[:,0:-tw_wall.n_vcoils]  
M_sc = M_sp_full[-tw_plasma.n_vcoils:, :].T  
M_sp = np.dot(mode_drive, M_sp_full[0:-tw_plasma.n_vcoils, :]).T

sM_s = compute_sM_s(M_sw, M_sp, M_sc, M_pw, M_cp, L_p, rho)
np.savez('sM_s_1by120_cn_n.npz', sM_s=sM_s)


# timescale is 10-100ms
tot_tpts = 1e4
tot_t = 3000*tot_tpts/1e4
t_vals = np.linspace(0, tot_t, int(tot_tpts))
#num_curr_modes = 1000000
eig_vecsT = eig_vecs.T

M_sw_full, M_sc_icoil, _ = tw_wall.compute_Msensor('floops.loc') 
M_sp_full, _, _ = tw_plasma.compute_Msensor('floops.loc')     # M_pc M_sc based on vcoils, icoils respectively?

M_sw = M_sw_full.T[:,0:-tw_wall.n_vcoils]  
M_sc = M_sp_full[-tw_plasma.n_vcoils:, :].T  
M_sp = np.dot(mode_drive, M_sp_full[0:-tw_plasma.n_vcoils, :]).T

sM_s = compute_sM_s(M_sw, M_sp, M_sc, M_pw, M_cp, L_p, rho)
np.savez('sM_s_1by120_pn_n.npz', sM_s=sM_s)



# current solve ...


(120,)
(1, 120, 6)
(1, 120, 6)
(1, 120, 6)
(1, 120, 6) 

(120,)
(1, 120, 6)
(1, 120, 6)
(1, 120, 6)


NameError: name 'tw_wall' is not defined

In [ ]:
    def build_reduced_model(self,basis_set,filename='---.h5',compute_B=False,sensor_obj=None):
        r'''! Build reduced model by projecting full model onto defined basis set of currents

        @param basis_set Basis set for projection `(nBasis,:)`
        @param filename Filename for saving reduced model
        @param compute_B Compute B-field reconstruction operators for reduced model?
        @param sensor_obj Sensor object to use
        @result Reduced model (see \ref ThinCurr_reduced)
        '''
        basis_set = numpy.ascontiguousarray(basis_set, dtype=numpy.float64)
        nbasis = c_int(basis_set.shape[0])
        sensor_ptr = c_void_p()
        if sensor_obj is not None:
            sensor_ptr = sensor_obj['ptr']
        cfilename = self._oft_env.path2c(filename)
        error_string = self._oft_env.get_c_errorbuff()
        if self.Lmat_hodlr:
            thincurr_reduce_model(self.tw_obj,cfilename,nbasis,basis_set,c_bool(compute_B),sensor_ptr,self.Lmat_hodlr,error_string)
        else:
            thincurr_reduce_model(self.tw_obj,cfilename,nbasis,basis_set,c_bool(compute_B),sensor_ptr,c_void_p(),error_string)
        if error_string.value != b'':
            raise Exception(error_string.value.decode())
        return ThinCurr_reduced(filename)

class ThinCurr_reduced:
    '''! Reduced ThinCurr thin-wall eddy current model class'''
    def __init__(self, filename):
        '''! Initialize Reduced ThinCurr object

        @param filename File containing reduced model
        '''
        with h5py.File(filename,'r') as file:
            mu0_scale = mu0 # Prior to addition of version flag magnetic units were saved for coils
            if 'ThinCurr_Version' in file:
                mu0_scale = 1.0 
            ## Current potential basis set
            self.Basis = numpy.asarray(file['Basis'])
            ## Self-inductance matrix for reduced model
            self.L = numpy.asarray(file['L'])
            ## Resistance matrix for reduced model
            self.R = numpy.asarray(file['R'])
            ## B-field reconstruction operator
            self.B = None
            if 'Bx' in file:
                self.B = [numpy.asarray(file['Bx']), numpy.asarray(file['By']), numpy.asarray(file['Bz'])]
            ## Model-sensor mutual inductance matrix 
            self.Ms = None
            if 'Ms' in file:
                self.Ms = numpy.asarray(file['Ms'])
            ## Model-coil mutual inductance matrix
            self.Mc = None
            ## B-field coil reconstruction operator
            self.Bc = None
            if 'Mc' in file:
                self.Mc = mu0_scale*numpy.asarray(file['Mc'])
                if 'Bx_c' in file:
                    self.Bc = mu0_scale*numpy.asarray([file['Bx_c'], file['By_c'], file['Bz_c']])
            ## Coil-sensor mutual inductance matrix
            self.Msc = None
            if 'Msc' in file:
                self.Msc = mu0_scale*numpy.asarray(file['Msc'])
    
    def reconstruct_potential(self,weights):
        r'''! Reconstruct full current potential on original grid

        @note To reconstruct the full current pass the output to @ref ThinCurr.reconstruct_current "reconstruct_current()"
        with the original model

        @param weights Reduced model basis weights
        @result Full current potential on original grid `(:)`
        '''
        return numpy.dot(weights,self.Basis)
    
    def reconstruct_Bfield(self,weights,coil_currs=None):
        '''! Reconstruct magnetic field on original grid

        @param weights Reduced model basis weights
        @param coil_currs Coil currents [A]
        @result Magnetic field on original grid `(:,3)`
        '''
        if (self.B is None):
            raise ValueError('Magnetic field reconstruction operator not part of this model')
        if coil_currs is not None:
            if (self.Bc is None):
                raise ValueError('Coil magnetic field reconstruction operator not part of this model')
            if coil_currs.shape[0] != self.Bc[0].shape[0]:
                raise ValueError('Size of "coil_currs" does not match number of coils in reduced model')
        Bfield = numpy.zeros((self.B[0].shape[1],3))
        for i in range(3):
            Bfield[:,i] = numpy.dot(weights,self.B[i])
            if self.Bc is not None:
                Bfield[:,i] += numpy.dot(coil_currs,self.Bc[i])
        return Bfield

    def get_eigs(self):
        '''! Compute eigenmodes for reduced model

        @result Eigenvalues `(:)`
        @result Eigenvectors `(:,:)`
        '''
        eig_vals, eig_vecs = numpy.linalg.eig(numpy.dot(numpy.linalg.inv(self.R),self.L))
        sort_inds = (-eig_vals).argsort()
        return eig_vals[sort_inds], eig_vecs[sort_inds,:]

    def run_td(self,dt,nsteps,coil_currs,status_freq=10,plot_freq=10):
        r'''! Perform a time-domain simulation

        @param dt Time step for simulation
        @param nsteps Number of steps to take
        @param coil_currs Current vs time array for Icoils `(:,n_icoils+1)` (first index is time)
        @param status_freq Frequency to print status information
        @param plot_freq Frequency to save plot files
        @result Sensor signals dictionary
          - `time` Timebase [s] `(:)`, 
          - `sensors` Sensor signals `(:,:)`
        @result Current history dictionary
          - `time` Timebase [s] `(:)`
          - `curr` Reduced model basis weights `(:,:)`,
          - `coil_curr` Coil currents [A / \f$ \mu_0 \f$ ] `(:,:)`
        '''
        Lforward = self.L - (dt/2.0)*self.R
        Lbackward = numpy.linalg.inv(self.L + (dt/2.0)*self.R)
        #
        vec_time = []
        vec_hist = []
        coil_hist = []
        sen_time = []
        sen_hist = []
        pot_tmp = numpy.zeros((self.L.shape[0],))
        t = 0.0
        print('Timestep {0} {1:12.4E} {2:12.4E}'.format(0,t,numpy.linalg.norm(pot_tmp)))
        vec_time.append(t)
        curr_tmp = numpy.zeros((coil_currs.shape[1]-1,))
        for j in range(coil_currs.shape[1]-1):
            curr_tmp[j] = numpy.interp(t,coil_currs[:,0],coil_currs[:,j+1],left=coil_currs[0,j+1],right=coil_currs[-1,j+1])
        coil_hist.append(curr_tmp)
        vec_hist.append(pot_tmp)
        if self.Ms is not None:
            sen_tmp = numpy.dot(pot_tmp,self.Ms)
            if self.Msc is not None:
                sen_tmp += numpy.dot(curr_tmp,self.Msc)
            sen_time.append(t)
            sen_hist.append(sen_tmp)
        #
        for i in range(nsteps):
            rhs = numpy.dot(pot_tmp,Lforward)
            if self.Mc is not None:
                dcurr_tmp = numpy.zeros((coil_currs.shape[1]-1,))
                for j in range(coil_currs.shape[1]-1):
                    dcurr_tmp[j] = numpy.interp(t+dt/4.0,coil_currs[:,0],coil_currs[:,j+1],left=coil_currs[0,j+1],right=coil_currs[-1,j+1])
                    dcurr_tmp[j] -= numpy.interp(t-dt/4.0,coil_currs[:,0],coil_currs[:,j+1],left=coil_currs[0,j+1],right=coil_currs[-1,j+1])
                    dcurr_tmp[j] += numpy.interp(t+dt*5.0/4.0,coil_currs[:,0],coil_currs[:,j+1],left=coil_currs[0,j+1],right=coil_currs[-1,j+1])
                    dcurr_tmp[j] -= numpy.interp(t+dt*3.0/4.0,coil_currs[:,0],coil_currs[:,j+1],left=coil_currs[0,j+1],right=coil_currs[-1,j+1])
                rhs -= numpy.dot(dcurr_tmp,self.Mc)
            pot_tmp = numpy.dot(rhs,Lbackward)
            t += dt
            if ((i+1) % status_freq) == 0:
                print('Timestep {0} {1:12.4E} {2:12.4E}'.format(i+1,t,numpy.linalg.norm(pot_tmp)))
            curr_tmp = numpy.zeros((coil_currs.shape[1]-1,))
            for j in range(coil_currs.shape[1]-1):
                curr_tmp[j] = numpy.interp(t,coil_currs[:,0],coil_currs[:,j+1],left=coil_currs[0,j+1],right=coil_currs[-1,j+1])
            if ((i+1) % plot_freq) == 0:
                vec_time.append(t)
                coil_hist.append(curr_tmp)
                vec_hist.append(pot_tmp)
            if self.Ms is not None:
                sen_tmp = numpy.dot(pot_tmp,self.Ms)
                if self.Msc is not None:
                    sen_tmp += numpy.dot(curr_tmp,self.Msc)
                sen_time.append(t)
                sen_hist.append(sen_tmp)
        #
        sensor_obj = {
            'time': numpy.array(sen_time),
            'sensors': numpy.array(sen_hist)
        }
        curr_obj = {
            'time': numpy.array(vec_time),
            'curr': numpy.array(vec_hist),
            'coil_curr': numpy.array(coil_hist)
        }
        return sensor_obj, curr_obj